In [24]:
from datetime import timedelta

import plotly.express as px
import plotly.subplots as sp
import polars as pl
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [25]:
calendar = pl.scan_csv("../data/calendar.csv", schema_overrides={"date": pl.Date()})
sales_train_validation = pl.scan_csv("../data/sales_train_validation.csv")
sell_prices = pl.scan_csv("../data/sell_prices.csv")

df = (
    # Flatten sales data
    sales_train_validation.unpivot(
        index=["id", "state_id", "store_id", "cat_id", "dept_id", "item_id"],
        variable_name="d",
        value_name="y",
    )
    .join(calendar, on="d", how="inner")
    .join(sell_prices, on=["store_id", "item_id", "wm_yr_wk"], how="inner")
    
    # Aggregate to date-state-item level
    .group_by("date", "state_id", "cat_id", "dept_id", "item_id", "weekday", "month")
    .agg(
        *(pl.first(col) for col in (
            "event_name_1", "event_type_1", "event_name_2","event_type_2", "snap_CA", "snap_TX", "snap_WI"
        )),
        pl.sum("y").alias("y"),
        pl.mean("sell_price").alias("sell_price"),
    )
    
    # Keep the 500 highest selling items
    .with_columns(pl.sum("y").over("item_id").alias("total_sales"))
    .with_columns(pl.col("total_sales").rank(method="dense", descending=True).alias("rank"))
    .filter(pl.col("rank") <= 500)
    .drop("total_sales", "rank")
    
    # Feature engineering
    .sort("date", "state_id", "item_id")
    # NOTE: we predict 28 days ahead, so lags should only be based on data
    # at least 28 days old, otherwise they are 'unknown' at inference time
    .with_columns(
        pl.col("y").shift(i).over("state_id", "item_id").alias(f"y_lag_{i}")
        for i in range(28, 35) # 4 to 5 weeks lagged
    )
    # Lags on store-item-weekday level
    .with_columns(
        pl.col("y").shift(i).over("state_id", "item_id", "weekday").alias(f"y_lag_dow_{i}")
        for i in range(4, 9) # 4 to 8 weeks lagged
    )
    # Moving averages on store-item level
    .with_columns(
        pl.col("y")
        .rolling_mean(window_size=period)
        .shift(lag)
        .over("state_id", "item_id")
        .alias(f"y_ma_{period}d_lag_{lag}")
        for period in (7, 14) # window sizes of 1 and 2 weeks
        for lag in (28, 35) # 4 and 5 weeks lagged
    )
    # Moving averages on store-item-weekday level
    .with_columns(
        pl.col("y")
        .rolling_mean(window_size=period)
        .shift(lag)
        .over("state_id", "item_id", "weekday")
        .alias(f"y_ma_dow_{period}d_lag_{lag}")
        for period in (4, 8) # window sizes of 4, and 8 weeks
        for lag in (4, 8) # 4 and 8 weeks lagged
    )
    .collect(engine="streaming")
)

In [ ]:
# Filter on sales below 50 for a cleaner plot
df_filtered = df.filter(pl.col("y") < 50)
sales_mean = df_filtered["y"].mean()
sales_median = df_filtered["y"].median()

# Plot distribution of demand
fig = px.histogram(
    df_filtered,
    x="y",
    nbins=100,
    title=f"Distribution of Demand (Mean: {sales_mean:.2f}, Median: {sales_median:.2f})",
    labels={"y": "Demand"},
)
fig.add_vline(x=sales_mean, line_dash="dash", line_color="red", annotation_text="Mean", annotation_position="top")
fig.add_vline(x=sales_median, line_dash="dash", line_color="green", annotation_text="Median", annotation_position="top")
fig.update_layout(height=400, width=800)
fig.show()

In [27]:
# Define features
categorical_features = [
    "state_id", "cat_id", "dept_id", "item_id", "weekday", "month",
    "event_name_1", "event_type_1", "event_name_2", "event_type_2",
    "snap_CA", "snap_TX", "snap_WI",
]
numerical_features = [
    "sell_price",
    *(f"y_lag_{i}" for i in range(28, 35)),
    *(f"y_lag_dow_{i}" for i in range(4, 9)),
    *(f"y_ma_{period}d_lag_{lag}" for period in (7, 14) for lag in (28, 35)),
    *(f"y_ma_dow_{period}d_lag_{lag}" for period in (4, 8) for lag in (4, 8)),
]

# Convert to pandas and set categorical features
df_pandas = df.to_pandas().astype({col: "category" for col in categorical_features})

# Split data (we predict last 28 days)
training_end_date = df_pandas["date"].max().date() - timedelta(days=28)
training_start_date = training_end_date - timedelta(days=3 * 365) # 3 years

training_data = df_pandas[
    (df_pandas["date"].dt.date >= training_start_date) &
    (df_pandas["date"].dt.date <= training_end_date)
]
validation_data = df_pandas[df_pandas["date"].dt.date > training_end_date]

In [39]:
# Plot time series of 4 random store-item combinations
store_item_filter_df = validation_data[["state_id", "item_id"]].drop_duplicates().sample(4)
store_item_filter = list(zip(store_item_filter_df["state_id"], store_item_filter_df["item_id"]))

fig = sp.make_subplots(rows=2, cols=2, subplot_titles=[f"{store} - {item}" for store, item in store_item_filter])
for i, (state_id, item_id) in enumerate(store_item_filter):
    df_filtered = validation_data[
        (validation_data["state_id"] == state_id)
        & (validation_data["item_id"] == item_id)
    ]
    fig.add_trace(px.line(df_filtered, x="date", y="y").data[0], row=i // 2 + 1, col=i % 2 + 1)
    fig.data[-1].update(name="Actual", line=dict(color="blue"), legendgroup="actual", showlegend=(i == 0))

fig.update_layout(height=600, width=1000, title_text="Time Series for 4 random state-item combinations")
fig.update_yaxes(range=[0, None])
fig.show()

In [42]:
# Train on MAE
model = LGBMRegressor(n_estimators=750, objective="mae")
model.fit(
    X=training_data[numerical_features + categorical_features],
    y=training_data["y"],
    categorical_feature=categorical_features,
)

y_pred = model.predict(validation_data[numerical_features + categorical_features])
validation_data_mae = validation_data.copy().assign(y_pred=y_pred)

# Evaluate MAE and MSE
print(f"Validation MAE: {mean_absolute_error(validation_data_mae['y'], y_pred):.4f}")
print(f"Validation MSE: {mean_squared_error(validation_data_mae['y'], y_pred):.4f}")

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032473 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5887
[LightGBM] [Info] Number of data points in the train set: 1640483, number of used features: 34
[LightGBM] [Info] Start training from score 9.000000
Validation MAE: 5.6391
Validation MSE: 89.1222


In [44]:
# Plot predictions per store-item combination
fig = sp.make_subplots(rows=2, cols=2, subplot_titles=[f"{store} - {item}" for store, item in store_item_filter])
for i, (state_id, item_id) in enumerate(store_item_filter):
    item_data = validation_data_mae[
        (validation_data_mae["state_id"] == state_id)
        & (validation_data_mae["item_id"] == item_id)
    ]
    row, col = i // 2 + 1, i % 2 + 1

    # Add actual values (blue)
    fig.add_trace(px.line(item_data, x="date", y="y").data[0], row=row, col=col)
    fig.data[-1].update(name="Actual", legendgroup="actual", showlegend=(i == 0))
    
    # Add predicted values (red)
    fig.add_trace(px.line(item_data, x="date", y="y_pred").data[0], row=row, col=col)
    fig.data[-1].update(name="Predicted", line=dict(color="red"), legendgroup="predicted", showlegend=(i == 0))

fig.update_layout(
    height=600, width=1000,
    title_text="Actuals vs Predictions for 4 random state-item combinations (MAE)",
    legend=dict(x=1, y=1, xanchor="left", yanchor="top")
)
fig.update_yaxes(range=[0, None])
fig.show()

In [50]:
# Plot aggregated predictions vs actuals
preds_item_date_mae = validation_data_mae.groupby("date").agg({"y": "sum", "y_pred": "sum"}).reset_index()
fig = px.line(
    preds_item_date_mae,
    x="date",
    y=["y", "y_pred"],
    title="Aggregated Actuals vs Predictions (MAE)",
    labels={"value": "Demand", "date": "Date", "variable": "Legend"},
)
fig.update_layout(height=400, width=1000)
fig.show()

In [43]:
# Train on MSE
model = LGBMRegressor(n_estimators=750, objective="mse")
model.fit(
    X=training_data[numerical_features + categorical_features],
    y=training_data["y"],
    categorical_feature=categorical_features,
)

y_pred = model.predict(validation_data[numerical_features + categorical_features])
validation_data_mse = validation_data.copy().assign(y_pred=y_pred)

# Evaluate MAE and MSE
print(f"Validation MAE: {mean_absolute_error(validation_data_mse['y'], y_pred):.4f}")
print(f"Validation MSE: {mean_squared_error(validation_data_mse['y'], y_pred):.4f}")

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037881 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5887
[LightGBM] [Info] Number of data points in the train set: 1640483, number of used features: 34
[LightGBM] [Info] Start training from score 14.772106
Validation MAE: 5.8126
Validation MSE: 88.4375


In [46]:
# Plot predictions per store-item combination
validation_data_mae_mse = validation_data_mae.merge(
    validation_data_mse[["date", "state_id", "item_id", "y_pred"]],
    on=["date", "state_id", "item_id"],
    suffixes=("_mae", "_mse"),
)
fig = sp.make_subplots(rows=2, cols=2, subplot_titles=[f"{store} - {item}" for store, item in store_item_filter])
for i, (state_id, item_id) in enumerate(store_item_filter):
    item_data = validation_data_mae_mse[
        (validation_data_mae_mse["state_id"] == state_id)
        & (validation_data_mae_mse["item_id"] == item_id)
    ]
    row, col = i // 2 + 1, i % 2 + 1
    
    # Add actual values (blue)
    fig.add_trace(px.line(item_data, x="date", y="y").data[0], row=row, col=col)
    fig.data[-1].update(name="Actual", legendgroup="actual", showlegend=(i == 0))
    
    # Add MAE predictions (red)
    fig.add_trace(px.line(item_data, x="date", y="y_pred_mae").data[0], row=row, col=col)
    fig.data[-1].update(name="Predicted (MAE)", line=dict(color="red"), legendgroup="predicted_mae", showlegend=(i == 0))
    
    # Add MSE predictions (orange)
    fig.add_trace(px.line(item_data, x="date", y="y_pred_mse").data[0], row=row, col=col)
    fig.data[-1].update(name="Predicted (MSE)", line=dict(color="orange"), legendgroup="predicted_mse", showlegend=(i == 0))

fig.update_layout(
    height=600, 
    width=1000, 
    title_text="Actuals vs Predictions for 4 random state-item combinations (MAE and MSE)",
    legend=dict(x=1, y=1, xanchor="left", yanchor="top")
)
fig.update_yaxes(range=[0, None])
fig.show()

In [48]:
# Plot both aggregated results side by side
preds_item_date_mse = validation_data_mse.groupby("date").agg({"y_pred": "sum"}).reset_index()
preds_compared = preds_item_date_mae.merge(
    preds_item_date_mse,
    on="date",
    suffixes=("_mae", "_mse"),
)

fig = px.line(
    preds_compared,
    x="date",
    y=["y", "y_pred_mae", "y_pred_mse"],
    title="Aggregated Actuals vs Predictions (MAE and MSE)",
    labels={"value": "Demand", "date": "Date", "variable": "Legend"},
)

# Update line colors for each trace
for trace in fig.data:
    if trace.name == "y":
        trace.line.color = "blue"
    elif trace.name == "y_pred_mae":
        trace.line.color = "red"
    elif trace.name == "y_pred_mse":
        trace.line.color = "orange"

fig.update_layout(height=400, width=1000)
fig.show()